Runs an extended Kalman filter on Prof Flaxman's SEIIR predictions and
measurement data for New York State from mid-April to 7 July 2020.

S - Susceptible
E - Exposed
I1 - Presymptomatic
I2 - Symptomatic
R - Recovered

In [ ]:
%load_ext autoreload

In [ ]:
%autoreload

In [ ]:
%reset

In [1]:
import math
import datetime

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

import data_sets_wDeaths
import seiir_compartmental

In [2]:
# functions to support the Kalman filtering
def get_predicts_prior(day, seiir):
    x_hat = np.array([[seiir['S'].loc[day]],
                      [seiir['E'].loc[day]],
                      [seiir['I1'].loc[day]],
                      [seiir['I2'].loc[day]],
                      [seiir['R'].loc[day]],
                      [10**-10]])

    beta_k = seiir['beta_pred'].loc[day]

    return x_hat, beta_k


def step_seiir(x_hat, constants, beta_k, days=7):
    s_dict = {'S': x_hat[0, 0],
              'E': x_hat[1, 0],
              'I1': x_hat[2, 0],
              'I2': x_hat[3, 0],
              'R': x_hat[4, 0]}

    s = pd.Series(s_dict)

    for i in range(days):
        infectious = s.loc['I1'] + s.loc['I2']
        s = seiir_compartmental.compartmental_covid_step(s, s.sum(),
                                                         infectious,
                                                         constants['alpha'],
                                                         beta_k,
                                                         constants['gamma1'],
                                                         constants['gamma2'],
                                                         constants['sigma'],
                                                         constants['theta'])
    x_hat_future_prior = np.array([[s.loc['S']],
                                   [s.loc['E']],
                                   [s.loc['I1']],
                                   [s.loc['I2']],
                                   [s.loc['R']],
                                   [x_hat[5,0]]])

    return x_hat_future_prior


def predict_step(x_hat_k1_prior, P, Q, beta_k, constants):
    S = x_hat_k1_prior[0, 0]
    E = x_hat_k1_prior[1, 0]
    I1 = x_hat_k1_prior[2, 0]
    I2 = x_hat_k1_prior[3, 0]
    R = x_hat_k1_prior[4, 0]
    N = S + E + I1 + I2 + R
    alpha = constants['alpha']
    sigma = constants['sigma']
    gamma1 = constants['gamma1']
    gamma2 = constants['gamma2']

    part_f_S = np.array([[-beta_k * math.pow(I1 + I2, alpha) / N],
                         [beta_k * math.pow(I1 + I2, alpha) / N],
                         [0],
                         [0],
                         [0],
                         [0]])

    part_f_E = np.array([[0],
                         [-sigma],
                         [sigma],
                         [0],
                         [0],
                         [0]])

    part_f_I1 = np.array([[-alpha * beta_k * S * math.pow(I1+I2, alpha-1) / N],
                          [alpha * beta_k * S * math.pow(I1+I2, alpha-1) / N],
                          [-gamma1],
                          [gamma1],
                          [0],
                          [0]])

    part_f_I2 = np.array([[-alpha * beta_k * S * math.pow(I1+I2, alpha-1) / N],
                          [alpha * beta_k * S * math.pow(I1+I2, alpha-1) / N],
                          [0],
                          [-gamma2],
                          [gamma2],
                          [0]])

    part_f_R = np.array([[0],
                         [0],
                         [0],
                         [0],
                         [0],
                         [0]])
    
    extra = np.array([[0],
                      [0],
                      [0],
                      [0],
                      [0],
                      [0]])

    # 6x6
    f_jacob = np.concatenate([part_f_S, part_f_E, part_f_I1, part_f_I2,
                              part_f_R, extra], axis=1)

    # 6x6
    # P_k1_prior = f_jacob * P * f_jacob^T + Q
    P_k1_prior = np.matmul(np.matmul(f_jacob, P), np.transpose(f_jacob)) + Q
    return P_k1_prior


def update_step(x_hat, x_hat_k1, P_k1, Rn, rho1, rho2, z_k):
    # 6x6
    ep = 10**-10
    H = np.array([[ep, 0, 0, 0, 0, 0],
                  [0, ep, 0, 0, 0, 0],
                  [0, 0, ep, rho1, 0, 0],
                  [0, 0, 0, rho2, 0, 0],
                  [0, 0, 0, 0, ep, 0],
                  [0, 0, 0, rho3, 0, ep]])
    #H = np.array([[0, 0, 0, 0, 0],
    #                  [0, 0, 0, 0, 0],
    #                  [0, 0, 0, rho1, 0],
    #                  [0, 0, 0, rho2, 0],
    #                  [0, 0, 0, 0, 0]])
    
    # Si = H * P_k1 * H^T + Rn
    Si = np.matmul(np.matmul(H, P_k1), np.transpose(H)) + Rn

    # K_new = P_k1 * H^T * Si^(-1)
    K_new = np.matmul(np.matmul(P_k1, np.transpose(H)), np.linalg.inv(Si))
    y_new = np.matmul(H, x_hat)

    # 6x1
    diff = z_k - y_new

    x_hat_k1_post = x_hat_k1 + np.matmul(K_new, diff)

    P_k1_post = P_k1 - np.matmul(np.matmul(K_new, Si), np.transpose(K_new))

    # joseph formulation
    #joe2 = np.matmul(np.matmul(K_new, Rn), np.transpose(K_new))
    #joe1 = np.eye(5) - np.matmul(K_new, H)
    #P_k1_post = np.matmul(np.matmul(joe1, P_k1), np.transpose(joe1)) - joe2

    return x_hat_k1_post, P_k1_post

def get_predict_measures():
    predictions = pd.read_csv(
        r'case_vs_symptom/kalman/predicted_measures/predictions.csv',
        header=None, names=['measure_date', 'prediction_date', 'prop', 'case',
                            'prop_pred7', 'case_pred7', 'prop_pred1',
                            'case_pred1'],
        index_col=False, usecols=[0, 4, 5])

    start_d = datetime.timedelta(days=int(predictions['measure_date'].iloc[0]))
    start = datetime.date(2019, 12, 31) + start_d
    end_d = datetime.timedelta(days=int(predictions['measure_date'].iloc[-1]))
    end = datetime.date(2019, 12, 31) + end_d

    measure_rng = pd.date_range(start=start, end=end)

    # code to save:
    # prediction_rng = pd.date_range(start=datetime.date(2019, 12, 31) +
    # datetime.timedelta(days=int(predictions['prediction_date'].iloc[0])),
    # end=datetime.date(2019, 12, 31) +
    # datetime.timedelta(days=int(predictions['prediction_date'].iloc[-1])))

    predictions.set_index(measure_rng, inplace=True)

    return predictions


def get_data_sets(state, fips=None):
    # the post hoc seiir model predictions provided by Prof Flaxman without
    # any kalman filtering:
    #seiir = pd.read_csv(r'data/seiir_compartments_post-hoc_ny_state_20200910.csv', header=0,
    #                    index_col='date', parse_dates=True)
    seiir = pd.read_csv(r'data/seiir_compartments_post-hoc_ny_state.csv', header=0,
                        index_col='date', parse_dates=True)
    

    fb_data = data_sets_wDeaths.create_symptom_df()
    fb_data_val = data_sets_wDeaths.create_symptom_df(valid=True)

    if fips is None:
        case_data = data_sets_wDeaths.create_case_df_state()
        case_data_geo = case_data.loc[state][['case_rate', 'death_rate']].copy()
        fb_data_geo = fb_data.loc[state].groupby('date').sum().copy()
        fb_data_val_geo = fb_data_val.loc[state].groupby('date').sum().copy()

    else:

        case_data = data_sets_wDeaths.create_case_df_county()
        case_data_geo = case_data.loc[fips][['case_rate', 'death_rate']].copy()

        if fips == 'New York City':
            nyc_fips = ['36005', '36061', '36047', '36085']
            fb_data_geo = fb_data.loc[(slice(None), '36081'), :].copy()
            fb_data_geo = fb_data_geo.mean(level='date')
            fb_data_val_geo = fb_data_val.loc[(slice(None), '36081'), :].copy()
            fb_data_val_geo = fb_data_val.mean(level='date')
            for borough in nyc_fips:
                fb_data_geo += fb_data.loc[(slice(None), borough), :].copy().mean(level='date')
                fb_data_val_geo += fb_data_val.loc[(slice(None), borough), :].copy().mean(level='date')
        else:
            fb_data_geo = fb_data.loc[(slice(None), fips), :].copy()
            fb_data_val_geo = fb_data_val.loc[(slice(None), fips), :].copy()
            # collapse down to a single index column (date)
            fb_data_geo = fb_data_geo.mean(level='date')
            fb_data_val_geo = fb_data_val_geo.mean(level='date')

    return seiir, fb_data_geo, fb_data_val_geo, case_data_geo


def calc_fb_ma7(fb_data):
    """
    Returns a Pandas series
    """
    # the fb_data is a DataFrame while the case_data is a Series
    fb_ma7 = fb_data.rolling(window=7).mean()
    fb_ma7 = fb_ma7.iloc[6:, :]
    prop_ma7 = fb_ma7['num_stl'].div(fb_ma7['n'])

    return prop_ma7, fb_ma7['n'].copy(), fb_ma7['num_stl'].copy()

In [3]:
# set constants
K0 = datetime.date(2020, 4, 12)

the_state = 'NY'
the_county = 'New York City'
#the_county = data_sets_wDeaths.get_fips('NY', 'Westchester')
constants = {
    'alpha': 0.948786,
    'gamma1': 0.500000,
    'gamma2': 0.662215,
    'sigma': 0.266635,
    'theta': 6.000000
    }

# set initial values for Kalman filter parameters
P_mult = 1
Q_mult = 1

# Rn is the R noise matrix; it remains constant thru the stepping of the
# Kalman filter
# prior to experiments, Rn_mult = 5*10**-4
Rn_mult = 5*10**-8

Rn_22 = 88
Rn_32 = 150

Rn_23 = 0
Rn_33 = 1

Rn_52 = 600

Rn = Rn_mult * np.array([[0, 0, 0, 0, 0, 0],
                         [0, 0, 0, 0, 0, 0],
                         [0, 0, Rn_22, Rn_23, 0, 0],
                         [0, 0, Rn_32, Rn_33, 0, 0],
                         [0, 0, 0, 0, 0, 0],
                         [0, 0, Rn_52, 0, 0, 0]])
Q = Q_mult * np.eye(6)
P = P_mult * np.eye(6)

# generate data
seiir, fb_data, fb_data_val, case_data = get_data_sets(
    data_sets.STATES[the_state], fips=the_county)

if the_county == 'New York City':
    county_pop = 0
    for each in ['36081', '36005', '36061', '36047', '36085']:
        this_count, state_pop = data_sets.get_pops(each)
        county_pop += this_count
else:
    county_pop, state_pop = data_sets.get_pops(the_county)

b = county_pop / state_pop

# calculate moving averages on the fb and case data
case_ma7 = case_data['case_rate'].rolling(window=7).mean()
case_ma7_all = case_ma7.iloc[6:]
case_data['death_rate'][case_data['death_rate'] < 0] = 0
deaths_ma28 = case_data['death_rate'].rolling(window=28).mean()
deaths_ma28_all = deaths_ma28.iloc[27:]

prop_ma7, n_ma7, num_stl_ma7 = calc_fb_ma7(fb_data)
prop_ma7_valid, n_ma7_valid, num_stl_ma7_valid = calc_fb_ma7(fb_data_val)


# get starting compartment values for the state level
x_hat_state_k0, beta_k0 = get_predicts_prior(K0, seiir)
# convert to county values:
x_hat_k0 = b * x_hat_state_k0
# set the death rate value in the county x vector
x_hat_k0[5,0] = deaths_ma28_all.loc[K0]
I2_county = x_hat_k0[3, 0]

rho1 = prop_ma7.loc['2020-04-12'] / I2_county
rho2 = case_ma7_all.loc['2020-04-12'] / I2_county
rho3 = 0.025 * rho2


# approximate rho values
# I2_county = b * I2
# prop_county = rho1 * I2_county
# case_county = rho2 * I2_county


# create empty dictionaries to hold the estimated values
prop_est = {}
case_est = {}
seiir_pred = {}
death_est = {}


x_death = deaths_ma28_all.loc[K0]

# Original data run ----------------
start = K0
d = start
while d <= datetime.date(2020, 9, 30):
    # each cycle of the while loop executes a step

    # get state level compartments
    x_hat_state_k, beta_k = get_predicts_prior(d, seiir)

    # step the state level compartments 7 days forward
    x_hat_state_k1 = step_seiir(x_hat_state_k, constants, beta_k)

    # convert the state level compartments to county level values
    x_hat_k = b * x_hat_state_k
    #x_hat_k[5, 0] = x_death
    x_hat_k1 = b * x_hat_state_k1
    #x_hat_k1[5, 0] = x_death

    # get measurements
    z_k = np.array([[0],
                    [0],
                    [prop_ma7.loc[d]],
                    [case_ma7_all.loc[d]],
                    [0],
                    [deaths_ma28_all.loc[d]]])

    # predict step
    P = predict_step(x_hat_k1, P, Q, beta_k, constants)
    print('step -----------')
    print('P:')
    print(P)

    # update step
    x_hat_post, P_post = update_step(x_hat_k, x_hat_k1, P, Rn,
                                     rho1, rho2, z_k)
    print('P_post:')
    print(P_post)
    print('x_hat_post:')
    print(x_hat_post)
    
    # store the posterior death value (we don't have an modeled estimate of it)
    #x_death = x_hat_post[5, 0]

    # store estimated values for proportion and case rate
    indexDate = d + datetime.timedelta(days=7)
    prop_est[indexDate] = rho1 * x_hat_post[3, 0]
    case_est[indexDate] = rho2 * x_hat_post[3, 0]
    seiir_pred[indexDate] = b * x_hat_k1[3, 0]
    death_est[indexDate] = x_hat_post[5, 0]

    # update the P and d
    P = P_post
    d += datetime.timedelta(days=1)
    
# create pandas series of the estimated case rate
predicted_case = pd.Series(case_est)
predicted_seiir_prior = pd.Series(seiir_pred)
predicted_death = pd.Series(death_est)

step -----------
P:
[[ 1.10876898 -0.10876898  0.1165979   0.03782786 -0.15442576  0.        ]
 [-0.10876898  1.17986321 -0.18769212 -0.03782786  0.15442576  0.        ]
 [ 0.1165979  -0.18769212  1.32109422 -0.25        0.          0.        ]
 [ 0.03782786 -0.03782786 -0.25        1.68852871 -0.43852871  0.        ]
 [-0.15442576  0.15442576  0.         -0.43852871  1.43852871  0.        ]
 [ 0.          0.          0.          0.          0.          1.        ]]
P_post:
[[ 1.78105086e-10 -9.27651567e-11  5.80846082e-09 -3.61157238e-08
  -1.04398148e-09 -3.35915992e-10]
 [-9.27651567e-11  7.42428341e-12 -5.28881272e-10  3.24884347e-09
   5.46581946e-11  2.86170215e-11]
 [-5.79300594e-10  1.52982904e-11  1.24180546e+00  3.18678767e-07
  -3.02886331e-11  8.71379539e-09]
 [ 3.97593724e-09 -1.66604404e-10  6.92496338e-10 -1.71596071e-09
  -5.28997068e-10 -3.79685216e-08]
 [-1.04398151e-09  5.46581946e-11 -7.98189319e-10  4.29059688e-09
   2.69419820e-10  4.03819368e-11]
 [ 1.25633361e-1

step -----------
P:
[[ 1.06234524e+00 -6.23452391e-02  1.37450086e-01 -1.37450039e-01
  -4.74156228e-08  0.00000000e+00]
 [-6.23452383e-02  1.06234524e+00 -1.37450082e-01  1.37450032e-01
   5.02484087e-08  0.00000000e+00]
 [ 1.37450135e-01 -1.37450135e-01  1.30303088e+00 -3.03030775e-01
  -1.07328772e-07  0.00000000e+00]
 [-1.37450155e-01  1.37450155e-01 -3.03030928e-01  1.30303082e+00
   1.04418945e-07  0.00000000e+00]
 [ 1.90694876e-08 -1.90006396e-08  4.20318302e-08 -4.21777185e-08
   1.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  1.00000000e+00]]
P_post:
[[-2.26714425e-10  1.14518714e-10 -2.02281071e-08 -2.11189349e-08
   8.57710790e-17 -1.85999277e-10]
 [ 1.14518617e-10 -2.32391884e-12  2.71031360e-08 -1.33136530e-10
  -5.38031358e-19 -9.77094969e-13]
 [-2.61045074e-10  1.36933520e-11  1.21224788e+00  3.12945926e-07
   7.69684281e-18  1.06766483e-08]
 [ 1.07969089e-09 -1.60799707e-11 -1.30174322e-07  2.28351782e

step -----------
P:
[[ 1.06139700e+00 -6.13969988e-02  1.36426935e-01 -1.36426888e-01
  -4.68832291e-08  0.00000000e+00]
 [-6.13969972e-02  1.06139700e+00 -1.36426932e-01  1.36426885e-01
   4.68357290e-08  0.00000000e+00]
 [ 1.36426983e-01 -1.36426983e-01  1.30314699e+00 -3.03146883e-01
  -1.04129590e-07  0.00000000e+00]
 [-1.36427004e-01  1.36427004e-01 -3.03147033e-01  1.30314693e+00
   1.04172969e-07  0.00000000e+00]
 [ 1.88782806e-08 -1.88798204e-08  4.19547190e-08 -4.19573002e-08
   1.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  1.00000000e+00]]
P_post:
[[ 8.21336332e-11 -3.52699953e-11 -2.57314927e-08  9.58062946e-09
  -3.17359489e-17  8.36245028e-11]
 [-3.52699536e-11 -1.15933929e-11  2.67855710e-08 -3.02131403e-09
  -4.04814251e-18 -2.49151730e-11]
 [ 9.49196277e-11  9.21335230e-12  1.21259431e+00  3.16073048e-07
  -3.64021674e-18  1.07174201e-08]
 [-4.00379813e-10 -4.72603623e-11 -1.27043171e-07 -5.40030909e

step -----------
P:
[[ 1.06214889e+00 -6.21488873e-02  1.37246758e-01 -1.37246711e-01
  -4.70505062e-08  0.00000000e+00]
 [-6.21488857e-02  1.06214889e+00 -1.37246754e-01  1.37246707e-01
   4.70634887e-08  0.00000000e+00]
 [ 1.37246806e-01 -1.37246806e-01  1.30308956e+00 -3.03089460e-01
  -1.03975720e-07  0.00000000e+00]
 [-1.37246827e-01  1.37246827e-01 -3.03089609e-01  1.30308951e+00
   1.04022843e-07  0.00000000e+00]
 [ 1.91051398e-08 -1.91055598e-08  4.21456679e-08 -4.20851425e-08
   1.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  1.00000000e+00]]
P_post:
[[ 1.99964489e-11  2.03764783e-11 -2.41261612e-08  1.19250071e-09
  -8.27228258e-18  1.07633171e-11]
 [ 2.03765269e-11 -6.07491835e-11  2.53271371e-08  4.79073461e-09
  -2.23273782e-17  4.25802770e-11]
 [ 1.92471317e-11  6.99109937e-11  1.21233866e+00  3.14322089e-07
   6.13609196e-19  1.06923321e-08]
 [-9.25192145e-11 -2.90802910e-10 -1.27205146e-07  1.21165877e

step -----------
P:
[[ 1.06218725e+00 -6.21872523e-02  1.37287528e-01 -1.37287481e-01
  -4.73942661e-08  0.00000000e+00]
 [-6.21872512e-02  1.06218725e+00 -1.37287524e-01  1.37287475e-01
   4.91735911e-08  0.00000000e+00]
 [ 1.37287577e-01 -1.37287577e-01  1.30308257e+00 -3.03082460e-01
  -1.06535852e-07  0.00000000e+00]
 [-1.37287598e-01  1.37287598e-01 -3.03082613e-01  1.30308251e+00
   1.04932294e-07  0.00000000e+00]
 [ 1.95755362e-08 -1.95614489e-08  4.30691043e-08 -4.29074248e-08
   1.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  1.00000000e+00]]
P_post:
[[ 4.75492978e-11 -2.44806675e-11 -2.35333195e-08 -4.86285545e-09
  -1.73051346e-17 -3.88367977e-11]
 [-2.44806467e-11  1.41220369e-12  2.69750069e-08  2.58531169e-10
  -3.26789282e-19  2.09325650e-12]
 [ 5.48972534e-11 -3.97060163e-12  1.21232338e+00  3.14289796e-07
   6.93375877e-20  1.06919886e-08]
 [-2.28095709e-10  9.13977227e-12 -1.30357943e-07  3.17292859e

...

step -----------
P:
[[ 1.09415655e+00 -9.41565527e-02  1.68201257e-01 -1.68201200e-01
  -5.75934202e-08  0.00000000e+00]
 [-9.41565499e-02  1.09415655e+00 -1.68201253e-01  1.68201196e-01
   5.66715088e-08  0.00000000e+00]
 [ 1.68201314e-01 -1.68201314e-01  1.30047481e+00 -3.00474711e-01
  -1.02451330e-07  0.00000000e+00]
 [-1.68201338e-01  1.68201338e-01 -3.00474855e-01  1.30047475e+00
   1.04020372e-07  0.00000000e+00]
 [ 2.14762177e-08 -2.13366116e-08  3.77369124e-08 -3.72293883e-08
   9.99999999e-01  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  1.00000000e+00]]
P_post:
[[ 1.42854617e-11 -9.86337401e-12 -2.94341680e-08  1.11206688e-09
  -3.34403314e-18  9.82092865e-12]
 [-9.86338788e-12  5.44186918e-12  3.21445790e-08 -4.78234119e-12
   6.09519615e-19 -2.56791497e-13]
 [ 1.39207812e-11 -6.02196071e-12  1.20182509e+00  3.12206693e-07
   1.19868397e-18  1.02760065e-08]
 [-6.05639983e-11  2.63775113e-11 -1.12748241e-07  3.49424711e

step -----------
P:
[[ 1.09386736e+00 -9.38673561e-02  1.67920063e-01 -1.67920005e-01
  -5.75542828e-08  0.00000000e+00]
 [-9.38673530e-02  1.09386735e+00 -1.67920059e-01  1.67920003e-01
   5.54435861e-08  0.00000000e+00]
 [ 1.67920119e-01 -1.67920119e-01  1.30039364e+00 -3.00393541e-01
  -1.00983032e-07  0.00000000e+00]
 [-1.67920143e-01  1.67920143e-01 -3.00393683e-01  1.30039358e+00
   1.03272213e-07  0.00000000e+00]
 [ 2.09518956e-08 -2.09633640e-08  3.73577397e-08 -3.71677870e-08
   1.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  1.00000000e+00]]
P_post:
[[-4.18383106e-11 -4.03303502e-11 -2.82179764e-08 -2.34695852e-09
   1.29559049e-17 -2.10384953e-11]
 [-4.03304057e-11  1.22498456e-10  3.52941517e-08 -1.83591758e-08
   3.75784976e-17 -1.57783691e-10]
 [-3.34800243e-11 -1.13512033e-10  1.20186000e+00  3.13855167e-07
  -2.26275587e-18  1.02912217e-08]
 [ 1.63489278e-10  4.72834494e-10 -1.10916370e-07 -7.53304308e

step -----------
P:
[[ 1.09294421e+00 -9.29442121e-02  1.67146688e-01 -1.67146631e-01
  -5.74556989e-08  0.00000000e+00]
 [-9.29442098e-02  1.09294421e+00 -1.67146684e-01  1.67146626e-01
   5.74974773e-08  0.00000000e+00]
 [ 1.67146745e-01 -1.67146745e-01  1.30058918e+00 -3.00589079e-01
  -1.03530150e-07  0.00000000e+00]
 [-1.67146768e-01  1.67146768e-01 -3.00589225e-01  1.30058912e+00
   1.03703991e-07  0.00000000e+00]
 [ 2.08775490e-08 -2.08752187e-08  3.73800987e-08 -3.71668101e-08
   1.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  1.00000000e+00]]
P_post:
[[ 7.51614326e-11 -3.89377836e-11 -2.98335375e-08  2.86361601e-09
  -2.23637874e-17  2.64630617e-11]
 [-3.89377419e-11  2.71427325e-12  3.21914053e-08 -4.03574646e-10
  -1.10511330e-19 -3.48936832e-12]
 [ 7.33544891e-11 -8.21173685e-12  1.20221571e+00  3.12740069e-07
  -1.86756206e-18  1.02955646e-08]
 [-2.95252767e-10  1.33931477e-11 -1.13763795e-07 -1.35860656e

step -----------
P:
[[ 1.09569407e+00 -9.56940718e-02  1.69533406e-01 -1.69533348e-01
  -5.84146762e-08  0.00000000e+00]
 [-9.56940694e-02  1.09569407e+00 -1.69533402e-01  1.69533344e-01
   5.84469191e-08  0.00000000e+00]
 [ 1.69533464e-01 -1.69533464e-01  1.30034865e+00 -3.00348550e-01
  -1.03506885e-07  0.00000000e+00]
 [-1.69533487e-01  1.69533487e-01 -3.00348696e-01  1.30034859e+00
   1.03456949e-07  0.00000000e+00]
 [ 2.10271840e-08 -2.10332671e-08  3.72715689e-08 -3.72831793e-08
   1.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  1.00000000e+00]]
P_post:
[[ 4.72863970e-11 -4.78851681e-12 -2.91323541e-08 -3.06812314e-09
  -1.49468890e-17 -2.41269409e-11]
 [-4.78847517e-12 -3.77089471e-11  3.11538831e-08  7.50235971e-09
  -1.09869239e-17  6.40113681e-11]
 [ 4.25315894e-11  3.27572969e-11  1.20134155e+00  3.10381640e-07
  -1.31307312e-18  1.02423638e-08]
 [-1.89075228e-10 -1.36885336e-10 -1.12918415e-07 -2.73303602e

step -----------
P:
[[ 1.09845385e+00 -9.84538474e-02  1.71889215e-01 -1.71889157e-01
  -5.88275128e-08  0.00000000e+00]
 [-9.84538449e-02  1.09845384e+00 -1.71889211e-01  1.71889152e-01
   5.88049769e-08  0.00000000e+00]
 [ 1.71889273e-01 -1.71889273e-01  1.30009911e+00 -3.00099011e-01
  -1.02672828e-07  0.00000000e+00]
 [-1.71889297e-01  1.71889297e-01 -3.00099154e-01  1.30009905e+00
   1.02679184e-07  0.00000000e+00]
 [ 2.10613018e-08 -2.10538916e-08  3.67755150e-08 -3.67991051e-08
   1.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  1.00000000e+00]]
P_post:
[[-1.12053478e-10  5.83370158e-11 -2.91326727e-08 -4.29149324e-09
   3.29139996e-17 -3.95878210e-11]
 [ 5.83369603e-11 -4.62008209e-12  3.29925431e-08 -3.60364932e-10
  -8.25751244e-19 -2.79283401e-12]
 [-1.06195636e-10  1.24120159e-11  1.20046606e+00  3.11310848e-07
   4.03783253e-18  1.02161020e-08]
 [ 4.32810038e-10 -2.65174827e-11 -1.11079037e-07 -6.98132663e

In [ ]:
deaths_ma28_all

In [ ]:
# MASE calculation -----------------

left = predicted_case.index[0]
right = predicted_case.index[-1]

# take error between prediction and actual
e = (case_ma7_all.loc[left:right] - predicted_case.loc[left:right]).abs()

case_ma7_diff = case_ma7_all.diff(7)
denom = (case_ma7_diff.loc[left:right]).abs()

term = e.div(denom)
term.replace([np.inf, -np.inf], np.nan, inplace=True)
term.dropna(inplace=True)
mase = term.sum() / len(term)
print('mase:', mase)

In [ ]:
# Not currently used ---------------
# Validation data run ---------------------
start = prop_ma7_valid.index[0]
d = start
while d <= prop_ma7_valid.index[-1]:
    # each cycle of the while loop executes a step

    # get state level compartments
    x_hat_state_k, beta_k = get_predicts_prior(d, seiir)

    # step the state level compartments 7 days forward
    x_hat_state_k1 = step_seiir(x_hat_state_k, constants, beta_k)

    # convert the state level compartments to county level values
    x_hat_k = b * x_hat_state_k
    x_hat_k1 = b * x_hat_state_k1

    # get measurements
    z_k = np.array([[0],
                    [0],
                    [prop_ma7_valid.loc[d]],
                    [case_ma7_all.loc[d]],
                    [0]])

    # predict step
    P = predict_step(x_hat_k1, P, Q, beta_k, constants)
    print('step -----------')
    print('P:')
    print(P)

    # update step
    x_hat_post, P_post = update_step(x_hat_k, x_hat_k1, P, Rn,
                                     rho1, rho2, z_k)
    print('P_post:')
    print(P_post)

    # store estimated values for proportion and case rate
    indexDate = d + datetime.timedelta(days=7)
    prop_est[indexDate] = rho1 * x_hat_post[3, 0]
    case_est[indexDate] = rho2 * x_hat_post[3, 0]
    seiir_pred[indexDate] = b * x_hat_k1[3, 0]

    # update the P and d
    P = P_post
    d += datetime.timedelta(days=1)
    
# create pandas series of the estimated case rate
predicted_case = pd.Series(case_est)
predicted_seiir_prior = pd.Series(seiir_pred)

In [4]:
# Plotting constants and variables ----------------
matplotlib.rcParams.update({'font.size': 20})
plt.style.use('seaborn-whitegrid')
purple = '#33016F'
gold = '#9E7A27'
gray = '#797979'
width = 4
%matplotlib qt

tick_end = predicted_seiir_prior.index[-1]

week_interval = pd.date_range(start=start, end=tick_end, freq='W')
week_interval = [x.to_pydatetime().date() for x in week_interval]

In [6]:
# Single plot with all lines

matplotlib.rcParams.update({'font.size': 20})
plt.style.use('seaborn-whitegrid')
tick_start = K0
tick_end = predicted_seiir_prior.index[-1]



fig3, ax31 = plt.subplots(1)
plt.sca(ax31)
plt.plot(case_ma7_all.loc[start:d].index, case_ma7_all.loc[start:d], label='Case Positive Rate', c=gold,
         linewidth=width)
plt.plot(predicted_seiir_prior.index, predicted_seiir_prior,
         label='IHME 7-day Forecast', c=gray, linewidth=width)
plt.plot(predicted_case.index, predicted_case, label='Our 7-Day Forecast',
         c=purple, linewidth=width)
plt.plot(predicted_death.index, predicted_death, label='Death Rate Forecast', c='blue', linewidth=width)
plt.plot(deaths_ma28_all.loc[start:d].index, deaths_ma28_all.loc[start:d], label='Death Rate', c='green', linewidth=width)
#plt.ylim(-2, 72)
plt.xticks(week_interval, rotation=30, ha='right', rotation_mode='anchor')
# plt.xlabel('Date')
plt.ylabel('Number of Cases per Day')
plt.legend(loc='upper left')

ax32 = ax31.twinx()
plt.sca(ax32)
plt.plot(prop_ma7.index, num_stl_ma7, c='red', label='Facebook Positive Symptoms',
         linewidth=width)
#plt.ylim(-.065, 2.5)
plt.grid(axis='y', linestyle=':')
plt.xticks(week_interval, rotation=30, ha='right', rotation_mode='anchor')

plt.ylabel('Number of Positive Symptom Response per Day')
plt.legend(loc='upper right')

plt.show()

In [ ]:
fig4, ax41 = plt.subplots(1)
plt.plot(predicted_death.index, predicted_death, label='Death Rate Forecast', c='blue')
plt.plot(deaths_ma28_all.loc[start:d].index, deaths_ma28_all.loc[start:d], label='Death Rate', c='green')
plt.legend()
plt.show()

In [ ]:
# not currently used ---------------------
# plot validation data

fig_v, ax_v = plt.subplots(1)
plt.sca(ax_v)


plt.plot(case_ma7_all.loc[start:end].index, case_ma7_all.loc[start:end], label='Case Positive Rate', c=gold,
         linewidth=width)
plt.plot(predicted_seiir_prior.index, predicted_seiir_prior,
         label='IHME 7-day Forecast', c=gray, linewidth=width)
plt.plot(predicted_case.index, predicted_case, label='Our 7-Day Forecast',
         c=purple, linewidth=width)
#plt.ylim(-2, 72)
plt.xticks(week_interval, rotation=30, ha='right', rotation_mode='anchor')
# plt.xlabel('Date')
plt.ylabel('Number of Cases per Day')
plt.legend(loc='upper left')

ax32 = ax_v.twinx()
plt.sca(ax32)
plt.plot(prop_ma7_valid.index, 100*prop_ma7_valid, c='red', label='Facebook Symptom Rate',
         linewidth=width)
plt.ylim(-.065, 2.5)
plt.grid(axis='y', linestyle=':')
plt.xticks(week_interval, rotation=30, ha='right', rotation_mode='anchor')

plt.ylabel('Percentage of Positive Symptom Response')
plt.legend(loc='upper right')

plt.show()

In [ ]:

# compute squared error for the overlapping time period
overlap = pd.date_range(start='2020-04-19', end='2020-07-07')
overlap_dt = [x.to_pydatetime().date() for x in overlap]

# error between the measured case rate (smoothed) and the
# SEIIR model scaled to the county (without any Kalman adjustment)
seiir_sq_err = 0

# error between the measured case rate (smoothed) and the predicted
# output of the Kalman filter
kalman_sq_err = 0

for day in overlap_dt:
    seiir_sq_err += (case_ma7_all[day] - predicted_seiir_prior[day])**2
    kalman_sq_err += (predicted_case[day] - case_ma7_all[day])**2

print('SSE between seiir forecast and case rate:', seiir_sq_err)
print('SSE between kalman forecast and case rate:', kalman_sq_err)


# plot findings, multiple plots ------------------
matplotlib.rcParams.update({'font.size': 20})
plt.style.use('seaborn-whitegrid')
tick_start = K0
tick_end = predicted_seiir_prior.index[-1]

week_interval = pd.date_range(start=tick_start, end=tick_end, freq='W')
week_interval = [x.to_pydatetime().date() for x in week_interval]

fig1, ax11 = plt.subplots(1)
plt.sca(ax11)
width = 4
plt.plot(case_ma7_all.loc[start:d].index, case_ma7_all.loc[start:d], label='Case Positive Rate', c=gold,
         linewidth=width)
plt.plot(predicted_seiir_prior.index, predicted_seiir_prior,
         label='IHME 7-day Forecast', c=gray, linewidth=width)
plt.ylim(-2, 72)
plt.xticks(week_interval, rotation=30, ha='right', rotation_mode='anchor')
plt.ylabel('Number of Cases per Day')
# plt.xlabel('Date')
plt.legend(loc='upper left')

fig2, ax21 = plt.subplots(1)
plt.sca(ax21)
plt.plot(case_ma7_all.loc[start:d].index, case_ma7_all.loc[start:d], label='Case Positive Rate', c=gold,
         linewidth=width)
plt.plot(predicted_seiir_prior.index, predicted_seiir_prior,
         label='IHME 7-day Forecast', c=gray, linewidth=width)
plt.plot(predicted_case.index, predicted_case, label='Our 7-Day Forecast',
         c=purple, linewidth=width)
plt.ylim(-2, 72)
plt.xticks(week_interval, rotation=30, ha='right', rotation_mode='anchor')
# plt.xlabel('Date')
plt.ylabel('Number of Cases per Day')
plt.legend(loc='upper left')

fig3, ax31 = plt.subplots(1)
plt.sca(ax31)
plt.plot(case_ma7_all.loc[start:d].index, case_ma7_all.loc[start:d], label='Case Positive Rate', c=gold,
         linewidth=width)
plt.plot(predicted_seiir_prior.index, predicted_seiir_prior,
         label='IHME 7-day Forecast', c=gray, linewidth=width)
plt.plot(predicted_case.index, predicted_case, label='Our 7-Day Forecast',
         c=purple, linewidth=width)
plt.ylim(-2, 72)
plt.xticks(week_interval, rotation=30, ha='right', rotation_mode='anchor')
# plt.xlabel('Date')
plt.ylabel('Number of Cases per Day')
plt.legend(loc='upper left')

ax32 = ax31.twinx()
plt.sca(ax32)
plt.plot(prop_ma7.index, 100*prop_ma7, c='red', label='Facebook Symptom Rate',
         linewidth=width)
plt.ylim(-.065, 2.5)
plt.grid(axis='y', linestyle=':')
plt.xticks(week_interval, rotation=30, ha='right', rotation_mode='anchor')

plt.ylabel('Percentage of Positive Symptom Response')
plt.legend(loc='upper right')


plt.show()